In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np 

In [2]:
#Loading csv containing all total number of votes form every state of the 2020 election data 
csv_file = "BPD_Part_1_Victim_Based_Crime_Data.csv"
crimedata = pd.read_csv(csv_file, encoding='ISO-8859-1')
crimedata.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
0,11/14/2020,08:50:00,9S,1600 FREDERICK AV,SHOOTING,Outside,FIREARM,711.0,WESTERN,FRANKLIN SQUARE,-76.642470,39.288348,NaN,STREET,Tri-District,1
1,11/14/2020,21:20:00,6C,500 W MULBERRY ST,LARCENY,O,NaN,121.0,CENTRAL,SETON HILL,-76.623351,39.293710,NaN,GAS STATION,NaN,1
2,11/14/2020,16:20:00,6C,3200 GREENMOUNT AVE,LARCENY,I,NaN,513.0,NORTHERN,ABELL,-76.609485,39.328073,NaN,RETAIL/SMALL BUSINES,NaN,1
3,11/14/2020,12:41:00,6C,3700 FALLS RD,LARCENY,I,NaN,531.0,NORTHERN,HAMPDEN,-76.635526,39.332721,NaN,DRUG STORE / MED BL,NaN,1
4,11/14/2020,14:19:00,4E,BALTIMORE ST & N WASHINGTON ST,COMMON ASSAULT,O,NaN,212.0,SOUTHEAST,BUTCHER'S HILL,-76.589158,39.291705,NaN,STREET,NaN,1


In [11]:
crimedata

,Longitude,Latitude,Description
0,-76.642470,39.288348,SHOOTING
1,-76.623351,39.293710,LARCENY
2,-76.609485,39.328073,LARCENY
3,-76.635526,39.332721,LARCENY
4,-76.589158,39.291705,COMMON ASSAULT
...,...,...,...
320759,-76.626880,39.328817,RAPE
320760,-76.691297,39.289582,RAPE
320761,-76.687168,39.326183,RAPE
320762,-76.635300,39.358898,RAPE


In [12]:
crimedata['Description'].value_counts()

LARCENY                 71332
COMMON ASSAULT          53921
BURGLARY                45407
LARCENY FROM AUTO       41673
AGG. ASSAULT            36010
AUTO THEFT              28064
ROBBERY - STREET        22304
ROBBERY - COMMERCIAL     5616
SHOOTING                 4459
ROBBERY - RESIDENCE      3375
ROBBERY - CARJACKING     2935
HOMICIDE                 2161
RAPE                     2141
ARSON                    1366
Name: Description, dtype: int64

In [13]:
crimedata_cleaned = crimedata.replace({ 'LARCENY FROM AUTO ':'LARCENY', 'ROBBERY - STREET':'ROBBERY' , 'ROBBERY - COMMERCIAL':'ROBBERY' , 'ROBBERY - RESIDENCE':'ROBBERY', 'ROBBERY - CARJACKING': 'ROBBERY'
}) 

In [14]:
crimedata_cleaned['Description'].value_counts()

LARCENY              71332
COMMON ASSAULT       53921
BURGLARY             45407
LARCENY FROM AUTO    41673
AGG. ASSAULT         36010
ROBBERY              34230
AUTO THEFT           28064
SHOOTING              4459
HOMICIDE              2161
RAPE                  2141
ARSON                 1366
Name: Description, dtype: int64

In [15]:
crimedata_cleaned = crimedata_cleaned.replace({ 'LARCENY FROM AUTO':'LARCENY'}) 

In [16]:
crimedata_cleaned['Description'].value_counts()

LARCENY           113005
COMMON ASSAULT     53921
BURGLARY           45407
AGG. ASSAULT       36010
ROBBERY            34230
AUTO THEFT         28064
SHOOTING            4459
HOMICIDE            2161
RAPE                2141
ARSON               1366
Name: Description, dtype: int64

In [17]:
crimedata_cleaned['id'] = crimedata_cleaned.index

In [18]:
crimedata_cleaned.set_index('id', inplace = True)

In [19]:
crimedata_cleaned

,Longitude,Latitude,Description
id,,,
0,-76.642470,39.288348,SHOOTING
1,-76.623351,39.293710,LARCENY
2,-76.609485,39.328073,LARCENY
3,-76.635526,39.332721,LARCENY
4,-76.589158,39.291705,COMMON ASSAULT
...,...,...,...
320759,-76.626880,39.328817,RAPE
320760,-76.691297,39.289582,RAPE
320761,-76.687168,39.326183,RAPE


In [ ]:
rds_connection_string = "postgres:Blue14horse@localhost:5432/election2020"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
conn = engine.connect()

In [ ]:
engine.table_names()

In [ ]:
crimedata_cleaned.to_sql(name = 'crime_data', con = engine, if_exists = 'append', index = True)

In [ ]:
crimedata_final = pd.read_sql('select * from crime_data' ,conn)

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
crimedata_final .to_csv("crimedata_final.csv", index=False, header=True)